In [1]:
import time
import pandas as pd
from scraper import Scrapper
from dataclasses import dataclass

In [2]:
@dataclass
class Etf:
    isin: str
    ticker: str
    name: str
    shares: int

In [3]:
etfs = [
      Etf('IE000OJ5TQP4', 'NATO', 'HANetf Future of Defence UCITS ETF', 40),
      Etf('IE00BFMXXD54', 'VUAA', 'Vanguard S&P 500 UCITS ETF', 30),
      Etf('IE00BMC38736', 'SMH', 'VanEck Semiconductor UCITS ETF', 25),
      Etf('IE000COQKPO9', 'ANAU', 'Invesco Nasdaq-100 ESG UCITS ETF', 20),
      Etf('IE00B53L3W79', 'CSSX5E', 'iShares Core EURO STOXX 50 UCITS ETF', 15),
      Etf('IE00BHZRQZ17', 'FLXI', 'Franklin FTSE India UCITS ETF', 10),
      Etf('IE00B4L5Y983', 'SWDA', 'iShares Core MSCI World UCITS ETF', 5)
]

In [4]:
df_data = pd.DataFrame(columns=['Data'] + [etf.ticker for etf in etfs])
df_holdings = pd.DataFrame(columns=['Holdings'] + [etf.ticker for etf in etfs])
df_countries = pd.DataFrame(columns=['Countries'] + [etf.ticker for etf in etfs])
df_sectors = pd.DataFrame(columns=['Sectors'] + [etf.ticker for etf in etfs])
df_realTimePrice = pd.DataFrame(columns=['Prices'] + [etf.ticker for etf in etfs])

In [5]:
for etf in etfs:
      scrap = Scrapper(etf.isin)
      data = scrap.catch_data('Data', etf.ticker)
      holdings = scrap.catch_holdings('Holdings', etf.ticker)
      countries = scrap.catch_countries('Countries', etf.ticker)
      sectors = scrap.catch_sectors('Sectors', etf.ticker)
      prices = scrap.real_time_data('Prices', etf.ticker)
      scrap.close()

      df_data = pd.concat([df_data, pd.DataFrame(data)], ignore_index=True)
      df_holdings = pd.concat([df_holdings, pd.DataFrame(holdings)], ignore_index=True)
      df_countries = pd.concat([df_countries, pd.DataFrame(countries)], ignore_index=True)
      df_sectors = pd.concat([df_sectors, pd.DataFrame(sectors)], ignore_index=True)
      df_realTimePrice = pd.concat([df_realTimePrice, pd.DataFrame(prices)], ignore_index=True)

del countries, data, holdings, prices, sectors, scrap

HANetf Future of Defence UCITS ETF | A3EB9T | IE000OJ5TQP4
Cookie consent accepted.
Clicked 6 times !
Driver was closed !
Vanguard S&P 500 UCITS ETF (USD) Accumulating | A2PFN2 | IE00BFMXXD54
Cookie consent accepted.
Clicked 6 times !
Driver was closed !
VanEck Semiconductor UCITS ETF | A2QC5J | IE00BMC38736
Cookie consent accepted.
Clicked 5 times !
Driver was closed !
Invesco Nasdaq-100 ESG UCITS ETF Acc | A3CZGT | IE000COQKPO9
Cookie consent accepted.
Clicked 6 times !
Driver was closed !
iShares Core EURO STOXX 50 UCITS ETF EUR (Acc) | A0YEDJ | IE00B53L3W79
Cookie consent accepted.
Clicked 7 times !
Driver was closed !
Franklin FTSE India UCITS ETF | A2PB5W | IE00BHZRQZ17
Cookie consent accepted.
Clicked 6 times !
Driver was closed !
iShares Core MSCI World UCITS ETF USD (Acc) | A0RPWH | IE00B4L5Y983
Cookie consent accepted.
Clicked 7 times !
Driver was closed !


In [6]:
def process_df(df, id_var, value_var='Data'):
      return (
            df.melt(id_vars=[id_var], var_name='Tickers', value_name=value_var)
            .dropna(subset=[value_var])
            .replace({r'\.': ',', '%': ''}, regex=True)
      )

In [7]:
pbi_holdings = process_df(df_holdings, 'Holdings')
pbi_sectors = process_df(df_sectors, 'Sectors')
pbi_countries = process_df(df_countries, 'Countries')
pbi_data = process_df(df_data, 'Data', 'Info')

del df_holdings, df_sectors, df_countries, df_data

In [8]:
df_realTimePrice.iloc[:, 1:] = df_realTimePrice.iloc[:, 1:].bfill(axis=1).ffill(axis=1)

df_realTimePrice = df_realTimePrice.iloc[:, :2]
df_realTimePrice.columns = ['Data', 'Tickers']

df_realTimePrice[['Currency', 'Price', 'ChangeEur', 'ChangePercent']] = df_realTimePrice['Data'].str.extract(r'(\w+)\s([\d\.]+)\s([\+\-]\d+\.\d+)\|([\+\-]\d+\.\d+%)')
df_realTimePrice = df_realTimePrice[['Tickers', 'Currency', 'Price', 'ChangeEur', 'ChangePercent']]

for etf in etfs:
      df_realTimePrice.loc[df_realTimePrice['Tickers'] == etf.ticker, 'Shares'] = etf.shares
      df_realTimePrice.loc[df_realTimePrice['Tickers'] == etf.ticker, 'Name'] = etf.name

del etf, etfs

In [9]:
df_realTimePrice = df_realTimePrice.fillna(0).replace({
    r'\.': ',',
    '%': ''
}, regex=True)

In [10]:
pbi_vola = pbi_data[pbi_data['Data'].str.contains('Volatility', regex=True)].replace({
    '-': '0',
    '%': '',
    'p,a,': ''
}, regex=True)

In [11]:
pbi_data = pbi_data[pbi_data['Data'].str.contains('Total expense', regex=True)].replace({
    '-': '0',
    '%': '',
    'p,a,': ''
}, regex=True)

In [12]:
with pd.ExcelWriter('ScrapedData.xlsx', engine='xlsxwriter') as writer:
    df_realTimePrice.to_excel(writer, sheet_name='RealPrice')
    pbi_holdings.to_excel(writer, sheet_name='Holdings')
    pbi_countries.to_excel(writer, sheet_name='Countries')
    pbi_sectors.to_excel(writer, sheet_name='Sectors')
    pbi_data.to_excel(writer, sheet_name='Info')
    pbi_vola.to_excel(writer, sheet_name='Volatility')

del writer